In [1]:
import numpy as np

In [13]:
res_path = "/home/handb/GeoSTHN/exper/abl_sthn/thgl-software-subset/output/run_0/thgl-software-subset_test_preds.npz"
res = np.load(res_path)
pred = res['preds']
label = res['labels']
from torchmetrics.classification import BinaryAUROC, BinaryAveragePrecision
import torch
auc_roc = BinaryAUROC()
auc_pr = BinaryAveragePrecision()

In [22]:
index = 0
length = pred.shape[0]
num = []
for i in range(length):
    pred_pos = pred[i, 0]
    pred_neg = pred[i, 1:]
    num_i = 0
    for neg in pred_neg:
        if pred_pos > neg:
            num_i += 1
    num.append(num_i)
print(np.mean(num))

8.808266666666666


In [24]:
# 统计字典
from collections import Counter
counter = Counter(num)
print(counter)

Counter({4: 665, 5: 649, 20: 648, 6: 632, 3: 558, 7: 541, 8: 488, 9: 483, 2: 454, 10: 384, 11: 319, 19: 256, 1: 237, 12: 226, 18: 170, 13: 160, 14: 148, 16: 144, 15: 131, 17: 126, 0: 81})


In [3]:
# ''' 使用torchmetrics计算AUC-ROC和AUC-PR '''


# print("AUC-ROC:", auc_roc(torch.tensor(pred), torch.tensor(label).long()).item())
# print("AUC-PR:", auc_pr(torch.tensor(pred), torch.tensor(label).long()).item())

In [4]:
# """ 按头节点分组 """
# # 获取label=1的索引
# pos_indices = np.where(label == 1)[0]
# s = [0]
# e = []
# for i in range(1, len(pos_indices)):
#     if pos_indices[i] - pos_indices[i-1] > 1:
#         e.append(pos_indices[i-1]+1)
#         s.append(pos_indices[i])
# e.append(pos_indices[-1]+1)
# all_edge = []
# for s, e in zip(s, e):
#     length = e - s
#     for i in range(s, e):
#         edge = [pred[i]]
#         for j in range(1,22):
#             edge.append(pred[i+length*(j)])
#         all_edge.append(edge)
# all_edge = np.array(all_edge)

In [5]:
# """ 分组后直接使用torchmetrics计算AUC-ROC和AUC-PR """
# edge_label = torch.zeros((21,1), dtype=torch.long)
# edge_label[0] = 1 
# all_exde_label = edge_label.unsqueeze(0).expand(7492, -1, -1)  # shape = (7492,21,1)

# auc = auc_roc(torch.tensor(all_edge), all_exde_label).item()
# print("AUC-ROC (all edges):", auc)
# ap = auc_pr(torch.tensor(all_edge), all_exde_label).item()
# print("AUC-PR (all edges):", ap)

In [6]:
# """ 分组后逐条计算AUC-ROC和AUC-PR的均值 """
# from tqdm import tqdm
# edge_label = torch.zeros((21,1), dtype=torch.long)
# edge_label[0] = 1 
# aucs = []
# aps = []
# for edge in tqdm(pred):
#     print(edge)
#     auc = auc_roc(torch.tensor(edge), edge_label).item()
#     aucs.append(auc) 
#     ap = auc_pr(torch.tensor(edge), edge_label).item()
#     aps.append(ap)
#     break
# print("mean AUC-ROC:", np.mean(aucs))
# print("std AUC-ROC:", np.std(aucs))
# print("mean AUC-PR:", np.mean(aps)) 
# print("std AUC-PR:", np.std(aps))

In [7]:
# """ 手动计算AUC-ROC和AUC-PR的函数 """
# def cal_auc(edge, edge_label):
#     pos = edge[edge_label==1]
#     neg = edge[edge_label==0]
#     # print("pos:", pos)
#     # print("neg:", neg)
#     n_prime = 0 # 记录 P > N 的次数
#     n_double_prime = 0 # 记录 P == N 的次数
#     n_pos = len(pos)
#     n_neg = len(neg)
#     for p in pos:
#         for n in neg:
#             if p > n:
#                 n_prime += 1
#             elif p == n:
#                 n_double_prime += 1
#     auc = (n_prime + 0.5 * n_double_prime) / (n_pos * n_neg)
#     return auc

# def cal_ap(edge, edge_label):
#     import numpy as np
#     scores = np.asarray(edge).flatten()
#     labels = np.asarray(edge_label).flatten().astype(int)
#     n_pos = (labels == 1).sum()
#     if n_pos == 0:
#         return 0.0
#     # 按分数从大到小排序，使用稳定排序以较好处理相同分数
#     order = np.argsort(-scores, kind="mergesort")
#     y = labels[order]
#     tp = np.cumsum(y == 1)
#     fp = np.cumsum(y == 0)
#     precision = tp / (tp + fp)
#     recall = tp / n_pos
#     # 在前面补一个 recall=0 和 precision=1 以匹配公式中的 R_{n-1}
#     recall = np.concatenate(([0.0], recall))
#     precision = np.concatenate(([1.0], precision))
#     ap = np.sum((recall[1:] - recall[:-1]) * precision[1:])
#     return float(ap)

In [8]:
# """ 使用自定义函数计算AUC-ROC和AUC-PR """
# auc = cal_auc(pred, label)
# ap = cal_ap(pred, label)
# print("AUC-ROC:", auc)
# print("AUC-PR:", ap)

In [9]:
# """ 使用自定义函数逐条计算AUC-ROC和AUC-PR的均值 """
# edge_label = torch.zeros((21,1), dtype=torch.long)
# edge_label[0] = 1 
# aucs = []
# aps = []
# for edge in tqdm(all_edge):
#     auc = cal_auc(edge, edge_label)
#     aucs.append(auc)
#     ap = cal_ap(edge, edge_label)
#     aps.append(ap)
# print("mean AUC-ROC:", np.mean(aucs))
# print("std AUC-ROC:", np.std(aucs))
# print("mean AUC-PR:", np.mean(aps)) 
# print("std AUC-PR:", np.std(aps))

In [10]:
pred.shape,label.shape
pred_pre, label_pre = pred[:, :2], label[:, :2]

In [11]:
print("AUC-ROC:", auc_roc(torch.tensor(pred_pre), torch.tensor(label_pre).long()).item())
print("AUC-PR:", auc_pr(torch.tensor(pred_pre), torch.tensor(label_pre).long()).item())

AUC-ROC: 0.4805784523487091
AUC-PR: 0.5118564367294312


In [12]:
""" 分组后逐条计算AUC-ROC和AUC-PR的均值 """
from tqdm import tqdm
aucs = []
aps = []

for i in tqdm(range(len(pred_pre))):
    edge = pred_pre[i]
    edge_label = label_pre[i]
    auc = auc_roc(torch.tensor(edge), torch.tensor(edge_label).long()).item()
    aucs.append(auc) 
    ap = auc_pr(torch.tensor(edge), torch.tensor(edge_label).long()).item()
    aps.append(ap)
print("mean AUC-ROC:", np.mean(aucs))
print("std AUC-ROC:", np.std(aucs))
print("mean AUC-PR:", np.mean(aps)) 
print("std AUC-PR:", np.std(aps))

  0%|          | 0/7500 [00:00<?, ?it/s]

100%|██████████| 7500/7500 [24:17<00:00,  5.14it/s]

mean AUC-ROC: 0.4382666666666667
std AUC-ROC: 0.4961743600344092
mean AUC-PR: 0.7191333333333333
std AUC-PR: 0.2480871800172046
